In [ ]:
from einops import rearrange

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

import numpy as np
import os
from pathlib import Path
from PIL import Image

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)), # resize the images to 224x24 pixels
    transforms.ToTensor(), # convert the images to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalize the images color channels
])

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, imgs_paths, idxs, transform):
        self.imgs_paths = np.array(imgs_paths)[idxs]
        self.transform = transform

    def __len__(self):
        return len(self.imgs_paths)

    def __getitem__(self, idx):
        # Load the image
        img_path = self.imgs_paths[idx]
        img = Image.open(img_path).convert('RGB')
        # Preprocess the image and send it to the chosen device ('cpu' or 'cuda')
        if self.transform:
            img = self.transform(img)
        return img

In [ ]:
from brainnet.config import get_cfg_defaults

cfg = get_cfg_defaults()

In [ ]:
subj = "subj01"

model_ckpt = '/home/admin/Algonaut/dat_colab_ckpt/{}/last.ckpt'.format(subj)
cfg.DATASET.DATA_DIR = "/home/admin/Algonaut/data/algonauts2023/{}".format(subj)
submission_dir = "/home/admin/Algonaut/submission"

challenge_images = np.loadtxt(f"/home/admin/Algonaut/data/ALG23/{subj}/image_ids/challenge_set.txt", dtype=int)

test_img_dir = '/home/admin/Algonaut/data/algonauts2023/{}/test_split/test_images'.format(subj)
test_imgs_paths = sorted(list(Path(test_img_dir).iterdir()))
test_img_list = os.listdir(test_img_dir)
idxs_test = np.arange(len(test_img_list))

In [ ]:
test_imgs_dataloader = DataLoader(
    ImageDataset(test_imgs_paths, idxs_test, transform),
    batch_size=6,
    shuffle=False
)

In [ ]:
from dat_models.dat import DAT
backbone = DAT()
backbone.load_state_dict(torch.load('/home/admin/Algonaut/dat_backbones/bkbn_upn_dat_b_160k.pth'))
cfg.MODEL.LAYERS = list(range(4))
cfg.MODEL.LAYER_WIDTHS = [128, 256, 512, 1024]
cfg.MODEL.BOTTLENECK_DIM = 128  # can be reduced to speed up

In [ ]:
from brainnet.plmodel import PLModel
import pytorch_lightning as pl


plmodel = PLModel(
    cfg, 
    backbone, 
    draw=False,  # draw on each epoch end
    cached=False,  # cache the features into cpu memory in first epoch
)
# plmodel.validation_epoch_end() is called on validation epoch to draw
plmodel = plmodel.cuda()

In [ ]:
plmodel.load_state_dict(torch.load(model_ckpt)['state_dict'])

In [ ]:
subj = 'subj01'
fmri_dir = "/home/admin2/Algonaut/data/algonauts2023/{}/training_split/training_fmri/".format(subj)
lh_fmri = np.load(os.path.join(fmri_dir, "lh_training_fmri.npy"))
rh_fmri = np.load(os.path.join(fmri_dir, "rh_training_fmri.npy"))
print(lh_fmri.shape, rh_fmri.shape)
lh_sample_submission = np.load('/home/admin2/Algonaut/algonauts_2023_challenge_submission/{}/lh_pred_test.npy'.format(subj))
rh_sample_submission = np.load('/home/admin2/Algonaut/algonauts_2023_challenge_submission/{}/rh_pred_test.npy'.format(subj))
print(lh_sample_submission.shape, rh_sample_submission.shape)
lh_sample_test = np.load('/home/admin2/Algonaut/data/alg23/submission/lgr/{}/lh_pred_test.npy'.format(subj))
rh_sample_test = np.load('/home/admin2/Algonaut/data/alg23/submission/lgr/{}/rh_pred_test.npy'.format(subj))
print(lh_sample_test.shape, rh_sample_test.shape)

In [ ]:
test_imgs_dataloader = DataLoader(
    ImageDataset(test_imgs_paths, idxs_test, transform),
    batch_size=3,
    shuffle=False
)


plmodel.eval()
outs = []
with torch.no_grad():
    for img in test_imgs_dataloader:
        img = img.cuda()
        out, _ = plmodel(img)
        outs.append(out.cpu().numpy())

In [ ]:
outs_np = np.vstack(outs)

fmri_dir = "/home/admin2/Algonaut/data/algonauts2023/{}/training_split/training_fmri/".format(subj)
lh_fmri = np.load(os.path.join(fmri_dir, "lh_training_fmri.npy"))
lh_fmri_len = lh_fmri.shape[1]

lh = outs_np[:,:lh_fmri_len]
rh = outs_np[:,lh_fmri_len:]

In [ ]:
lh_fmri_test_pred = lh.astype(np.float32)
rh_fmri_test_pred = rh.astype(np.float32)
np.save(os.path.join(submission_dir,'{}/lh_pred_test.npy'.format(subj)), lh_fmri_test_pred)
np.save('/home/admin2/Algonaut/data/alg23/submission/lgr_dat/{}/rh_pred_test.npy'.format(subj), rh_fmri_test_pred)
print('/home/admin2/Algonaut/data/alg23/submission/lgr_dat/{}/rh_pred_test.npy'.format(subj))